In [6]:
import cv2
import numpy as np
import base64
from cryptography.fernet import Fernet

# Function to decrypt the message
def decrypt_message(encrypted_message, key):
    cipher = Fernet(key)
    return cipher.decrypt(encrypted_message).decode()

# Function to extract binary data from the image
def extract_binary_data(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Error: Unable to read image at '{image_path}'. Check the file path and format.")

    binary_data = ""
    for row in img:
        for pixel in row:
            for channel in pixel:  # Iterate through R, G, B channels
                binary_data += str(channel & 1)  # Extract the least significant bit
                
    return binary_data

# Function to convert binary data back to bytes
def binary_to_bytes(binary_data):
    bytes_list = [binary_data[i:i+8] for i in range(0, len(binary_data), 8)]
    return bytearray([int(byte, 2) for byte in bytes_list])

# Function to extract and decrypt the message
def decode_message(image_path, key):
    binary_data = extract_binary_data(image_path)
    
    # Find the end marker (1111111111111110)
    end_marker = "1111111111111110"
    end_index = binary_data.find(end_marker)
    if end_index == -1:
        raise ValueError("Error: No hidden message found in the image.")
    
    binary_message = binary_data[:end_index]  # Extract only the message part
    encrypted_message = binary_to_bytes(binary_message)
    
    # Convert key from Base64 string to bytes
    try:
        key_bytes = base64.urlsafe_b64decode(key)
    except Exception as e:
        raise ValueError(f"Invalid encryption key. Ensure you entered the correct key.\nError: {e}")
    
    # Decrypt the message
    decrypted_message = decrypt_message(encrypted_message, key_bytes)
    return decrypted_message

# Input values in Jupyter Notebook
image_path = input("Enter the path to the stego image: ").strip()
key = input("Enter the encryption key: ").strip()

# Decode and display the message
try:
    decoded_message = decode_message(image_path, key)
    print(f"\nDecoded Message: {decoded_message}")
except Exception as e:
    print(f"\nError: {e}")


Enter the path to the stego image:  C:\Users\user\Downloads\Nature pic\stego_image.png
Enter the encryption key:  LV0cqv01VCewhkjvyIaoshVgRYOYMk5HsBVpySQB6Kk=



Error: Fernet key must be 32 url-safe base64-encoded bytes.
